In [148]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import maketab as mt

norm_upper = 4
norm_lower = 2

In [149]:
#real data

def norm(signal):
    signal = (signal - norm_lower) / (norm_upper - norm_lower)
    return signal

def load_data(path_dir='data/5-2-25/'):
    t, signal = mt.battery(path_dir)
    tleft = 1 - t / max(t)
    return norm(signal[50:500])

#plt.plot(load_data())
#plt.show()

In [ ]:
#Testing with artificial data
def generate_data(num_points=500, noise_std=0.1):
    x = np.arange(num_points)
    y = np.exp(x / 200) + 0.3*np.sin(x/20)  # Exponential + sinewave harmonics
    y += np.random.normal(scale=noise_std, size=num_points)  # Add noise

    scaler = MinMaxScaler(feature_range=(-1, 1))
    y = scaler.fit_transform(y.reshape(-1, 1)).flatten()
    print(y.shape)
    return y

#plt.plot(generate_data())
#plt.show()

In [150]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_length):
        self.data = data
        self.seq_length = seq_length
        self.sequences, self.targets = self.create_sequences()
    
    def create_sequences(self):
        sequences, targets = [], []
        for i in range(len(self.data) - self.seq_length):
            sequences.append(self.data[i:i + self.seq_length])
            targets.append(self.data[i + self.seq_length])
        return np.array(sequences), np.array(targets)
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return torch.tensor(self.sequences[idx], dtype=torch.float32), torch.tensor(self.targets[idx], dtype=torch.float32)

In [151]:
class ElmanRNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=100, output_size=1, num_layers=2):
        super(ElmanRNN, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Take last time step's output
        return out

In [152]:
def train_model(model, train_loader, epochs=100, lr=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        total_loss = 0
        for batch_x, batch_y in train_loader:
            batch_x = batch_x.unsqueeze(-1)  # Add feature dimension
            optimizer.zero_grad()
            output = model(batch_x)
            loss = criterion(output.squeeze(), batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {total_loss / len(train_loader):.4f}')

In [157]:
def recursive_forecast(model, data, seq_length, start_index, forecast_length, theta=100):
    model.eval()
    forecast = data[start_index:start_index + seq_length]  # Start with true data
    
    for i in range(forecast_length):
        seq = torch.tensor(forecast[-seq_length:], dtype=torch.float32).unsqueeze(0).unsqueeze(-1)
        pred = model(seq).item()
        forecast.append(pred)

        # Every `theta` steps, reset by using actual data
        if (i + 1) % theta == 0 and start_index + len(forecast) < len(data):
            forecast[-seq_length:] = data[start_index + len(forecast) - seq_length : start_index + len(forecast)]

    return forecast

In [158]:
data = load_data()
seq_length = 90
batch_size = 16  # Adjust for performance
seq_length_test = 30
theta = 90
total_length = len(data)
interval_length = total_length // 3
model = ElmanRNN(hidden_size=50, num_layers=3)

In [155]:
dataset = TimeSeriesDataset(data, seq_length)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
train_model(model, train_loader, epochs=100, lr=0.001)

Epoch 0, Loss: 0.3145
Epoch 10, Loss: 0.0002
Epoch 20, Loss: 0.0002
Epoch 30, Loss: 0.0002
Epoch 40, Loss: 0.0002
Epoch 50, Loss: 0.0002
Epoch 60, Loss: 0.0002
Epoch 70, Loss: 0.0002
Epoch 80, Loss: 0.0002
Epoch 90, Loss: 0.0002


In [159]:
forecast_1 = recursive_forecast(model, data, seq_length_test, 0, interval_length, theta=theta)

forecast_2 = recursive_forecast(model, data, seq_length_test, interval_length, interval_length, theta=theta)

forecast_3 = recursive_forecast(model, data, seq_length_test, 2*interval_length, interval_length, theta=theta)

# Plot results
plt.plot(range(len(data)), data, label='Original Data')
plt.plot(range(interval_length), forecast_1[:interval_length], label='Forecast 1st third', linestyle='dashed')
plt.plot(range(interval_length, 2 * interval_length), forecast_2[:interval_length], label='Forecast 2nd third', linestyle='dashed')
plt.plot(range(2 * interval_length, 3 * interval_length), forecast_3[:interval_length], label='Forecast 3nd third', linestyle='dashed')
plt.legend()
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'append'